In [ ]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Transbot_Lib import Transbot
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
# 创建Transbot对象 bot
# Create Transbot object as bot
bot = Transbot()


In [ ]:
# 启动接收数据，只能启动一次，所有读取数据的功能都是基于此方法
# Start to receive data, can only start once, all read data function is based on this method  
bot.create_receive_threading()

## 自动上报数据开关

In [ ]:
# 开启自动发送数据
# enable=True，底层扩展板会每隔40毫秒发送一次数据。enable=False，则不发送。
# forever=True永久保存，=False临时作用。
# Enable automatic data sending  
# enable=True, the underlying expansion board will send data every 40 milliseconds.  If enable=False, the port is not sent.  
# forever=True for permanent, =False for temporary.  
enable = True
bot.set_auto_report_state(enable, forever=False)

In [ ]:
# 关闭自动发送数据
# enable=True，底层扩展板会每隔40毫秒发送一次数据。enable=False，则不发送。
# forever=True永久保存，=False临时作用。
# Disable automatic sending of data  
# enable=True, the underlying expansion board will send data every 40 milliseconds.  If enable=False, the port is not sent.  
# forever=True for permanent, =False for temporary.  
enable = False
bot.set_auto_report_state(enable, forever=False)

In [ ]:
# 清除单片机自动发送过来的缓存数据
# Clear the cache data automatically sent by the MCU
bot.clear_auto_report_data()

## 小车控制速度相关参数设置
## Set parameters related to car control speed

In [ ]:
# 控制电机运动
# Control motor movement
def car_motion(line, angular):
    # 为了避免电机死区，这里增加了判断条件。
    # In order to avoid motor dead zone, the judgment condition is added here
    if abs(line) >= 5 or abs(angular) >= 10:
        speed_l = line / 100.0
        speed_a = angular / 100.0
        bot.set_car_motion(speed_l, speed_a)
        return speed_l, speed_a
    else:
        bot.set_car_motion(0, 0)
        return 0, 0

# 创建两个滑块来控制小车的线速度和角速度
# Create two sliders to control the linear and angular velocity of the car
interact(car_motion, line=widgets.IntSlider(min=-45,max=45,step=1,value=0), \
         angular=widgets.IntSlider(min=-200,max=200,step=1,value=0));


In [ ]:
# 停止运动 
# stop motion
bot.set_car_motion(0, 0)

In [ ]:
# 获取小车线速度和角速度数据
# Get the linear velocity and angular velocity data of the car
try:
    while True:
        v, a = bot.get_motion_data()
        print("speed:", v, a)
        bot.clear_auto_report_data()
        time.sleep(.1)
except KeyboardInterrupt:
    pass

In [ ]:
# 设置小车运动最小速度限制值（最小值）。默认情况下可不设置。
# 举个例子：当设置了线速度限制值为0.1时，下次调用set_car_motion(0.05, 0)函数时，单片机判断0.05小于0.1，自动把输入0.05设置为0.1，如果是set_car_motion(-0.05, 0)，则会把-0.05设置为-0.1。角速度限制值也是同样的功能。
# line_limit=[0, 0.20]，angular_limit=[0, 1.00]
# forever=True永久保存，=False临时作用。
# Set the minimum speed limit value (min) of the car movement.  This parameter is optional by default.  
# Here's an example:  When set_CAR_motion (0.05, 0) is called next time, the microcontroller determines that 0.05 is less than 0.1 and automatically sets the input 0.05 to 0.1. If set_car_motion(-0.05, 0) is used,  Sets -0.05 to -0.1.  The angular velocity limit also serves the same function.  
# line_limit=[0, 0.20], angular_limit=[0, 1.00]  
# forever=True for permanent, =False for temporary.  
line_limit = 0.1
angular_limit = 0.5
bot.set_speed_limit(line_limit, angular_limit, forever=False)

## PID相关参数设置

In [ ]:
# PID 参数控制，会影响set_car_motion函数控制小车的运动速度变化情况。默认情况下可不调整。
# PID parameter control will affect the set_CAR_motion function to control the speed change of the car.  This parameter is optional by default.
kp = 0.1
ki = 0.0
kd = 4.0
bot.set_pid_param(kp, ki, kd, forever=False)

In [ ]:
kp, ki, kd = bot.get_motion_pid()
print("PID:", kp, ki, kd)

## 陀螺仪相关参数设置
## Set parameters related to gyroscope

In [ ]:
# 开启陀螺仪调节方向。
# 由于小车振动以及陀螺仪自身漂移的问题，所以开启陀螺仪调节方向后，不能长时间一直运动，否则会陀螺仪会对小车运动造成干扰，导致一个轮子快一个轮子慢。
# 长时间测试时请将此功能关闭。
# enable=True开启，=False关闭。
# forever=True永久保存，=False临时作用。此参数最好不要调节，保持默认即可。
# Due to the vibration of the car and the drift of the gyroscope itself, it is not allowed to keep moving for a long time after turning on the gyroscope to adjust the direction, 
# otherwise the gyroscope will cause interference to the car movement, resulting in one wheel faster and the other wheel slower.  
# Please turn this feature off for long time test.  
# forever=True for permanent, =False for temporary.  This parameter is best not to adjust, keep the default.
enable = True
bot.set_imu_adjust(enable, forever=False)

In [ ]:
# 关闭陀螺仪调节方向。
# enable=True开启，=False关闭。
# forever=True永久保存，=False临时作用。此参数最好不要调节，保持默认即可。
# Turn off gyroscope and adjust direction.   
# forever=True for permanent, =False for temporary.  This parameter is best not to adjust, keep the default.
enable = False
bot.set_imu_adjust(enable, forever=False)

In [ ]:
state = bot.get_imu_state()
print("imu state:", state)

In [ ]:
# 获取加速度计三轴数据
# Get accelerometer triaxial data
try:
    while True:
        a_x, a_y, a_z = bot.get_accelerometer_data()
        print("acce:", a_x, a_y, a_z)
        time.sleep(.1)
except KeyboardInterrupt:
    pass

In [ ]:
# 获取陀螺仪三轴数据
# Get the data of three axis of gyroscope
try:
    while True:
        g_x, g_y, g_z = bot.get_gyroscope_data()
        print("gyro:", g_x, g_y, g_z)
        time.sleep(.1)
except KeyboardInterrupt:
    pass

## 恢复出厂配置，重置Flash数据。
## Restore factory Settings and reset Flash data

In [ ]:
# 恢复出厂配置 
# Restoring factory Settings
bot.reset_flash_value()

In [ ]:
# 程序结束后请删除transbot对象，避免在其他程序中使用transbot库造成冲突
# After the program is complete, delete the Transbot object to avoid conflicts caused by using the Transbot library in other programs
del bot